对于上面的模型示例，每个神经层都对输入数据进行如下变换

$output = relu(dot(input, W) + b)$

在这个表达式中，W和b是张量，均为该层的属性。它们被称为该层的权重（weight）或可训练参数（trainable parameter），分别对应属性kernel和bias。这些权重包含模型从训练数据中学到的信息。

一开始，这些权重矩阵取较小的随机值，这一步叫作随机初始化（random initialization）。当然，W和b都是随机的，relu(dot(input, W) + b)不会得到任何有用的表示。虽然得到的表示没有意义，但这是一个起点。下一步则是根据反馈信号逐步调节这些权重。这个逐步调节的过程叫作训练（training），也就是机器学习中的“学习”过程。

上述过程发生在一个训练循环（training loop）内，其具体流程如下。在一个循环中重复下列步骤，直到损失值变得足够小。
1. 抽取训练样本x和对应目标y_true组成的一个数据批量。
2. 在x上运行模型［这一步叫作前向传播（forward pass）］，得到预测值y_pred。
3. 计算模型在这批数据上的损失值，用于衡量y_pred和y_true之间的差距。
4. 更新模型的所有权重，以略微减小模型在这批数据上的损失值。

最终得到的模型在训练数据上的损失值非常小，即预测值y_pred与预期目标y_true之间的差距非常小。模型已“学会”将输入映射到正确的目标。

**简化理解：**

第1步看起来很简单，只是输入/输出（I/O）的代码。第2步和第3步仅仅是应用了一些张量运算。难点在于第4步：更新模型的权重。对于模型的某个权重系数，你怎么知道这个系数应该增大还是减小，以及变化多少？

一种简单的解决方案是，保持模型的其他权重不变，只考虑某一个标量系数，让其尝试不同的取值。假设这个系数的初始值为0.3。对一批数据做完前向传播后，模型在这批数据上的损失值是0.5。如果将这个系数改为0.35并重新运行前向传播，那么损失值增大为0.6。但如果将这个系数减小到0.25，那么损失值减小为0.4。在这个例子中，将系数减小0.05似乎有助于让损失值最小化。对于模型的所有系数都要重复这一过程。

但这种方法非常低效，因为系数有很多（通常有上千个，有时甚至多达上百万个），对每个系数都要计算两次前向传播，计算代价很大。

**梯度下降法（gradient descent）**

梯度下降是驱动现代神经网络的优化方法，其要点如下。我们的模型用到的所有函数（比如dot或+），都以一种平滑、连续的方式对输入进行变换。举个例子，对于z = x + y，y的微小变化只会导致z的微小变化。如果你知道y的变化方向，就可以推断出z的变化方向。用数学语言来讲，这些函数是**可微（differentiable）**的。将这样的函数组合在一起，得到的函数仍然是可微的。尤其是，将模型系数映射到模型在数据批量上损失值的函数，也是可微的：模型系数的微小变化，将导致损失值发生可预测的微小变化。我们可以用一个叫作**梯度（gradient）**的数学运算符来描述：模型系数向不同方向移动时，损失值如何变化。计算出梯度，就可以利用它来更新系数，使损失值减小（在一次更新中全部完成，而不是一次更新一个系数）。

## 2.4.1 导数
斜率a被称为f在p点的导数（derivative）。如果a为负，那么说明x在p点附近的微增将导致f(x)减小（如图2-17所示）；如果a为正，那么x的微增将导致f(x)增大。此外，a的绝对值（导数大小）表示这种增大或减小的速度。
![f在p点的导数](image/f在p点的导数.png)

## 2.4.2 张量运算的导数：梯度
导数这一概念可以应用于任意函数，只要函数所对应的表面是连续且光滑的。张量运算（或张量函数）的导数叫作梯度（gradient）。梯度就是将导数这一概念推广到以张量为输入的函数。张量函数的梯度表示该函数所对应多维表面的曲率（curvature）。它表示的是，当输入参数发生变化时，函数输出如何变化。

一个机器学习中的例子，假设我们有：
* 一个输入向量x（数据集中的一个样本）；
* 一个矩阵W（模型权重）；
* 一个目标值y_true（模型应该学到的与x相关的结果）；
* 一个损失函数loss（用于衡量模型当前预测值与y_true之间的差距）。

我们用W来计算预测值y_pred，然后计算损失值，即预测值y_pred与目标值y_true之间的差距。

``
y_pred = dot(W, x) # 利用模型权重W对x进行预测
loss_value = loss(y_pred, y_true) # 估算预测值的偏差有多大
``

现在我们想利用梯度来更新W，以使loss_value变小。如何做到这一点呢？如果输入数据x和y_true保持不变，那么可以将前面的运算看作一个将模型权重W的值映射到损失值的函数。

``
loss_value = f(W) # f描述的是：当W变化时，损失值所形成的曲线（或高维表面）
``

## 2.4.3 随机梯度下降
给定一个可微函数，理论上可以用解析法找到它的最小值：函数的最小值就是导数为0的点，因此只需找到所有导数为0的点，然后比较函数在其中哪个点的取值最小。

将这一方法应用于神经网络，就是用解析法求出损失函数最小值对应的所有权重值。可以通过对方程grad(f(W), W) = 0求解W来实现这一方法。这是一个包含N个变量的多项式方程，其中N是模型的系数个数。当N = 2或N = 3时，可以对这样的方程进行求解，但对于实际的神经网络是无法求解的，因为参数的个数不会少于几千个，而且经常有上千万个。

不过我们可以基于当前在随机数据批量上的损失值，一点一点地对参数进行调节。我们要处理的是一个可微函数，所以可以计算出它的梯度，从而有效地实现第4步。沿着梯度的反方向更新权重，每次损失值都会减小一点。
1. 抽取训练样本x和对应目标y_true组成的一个数据批量。
2. 在x上运行模型，得到预测值y_pred。这一步叫作前向传播。
3. 计算模型在这批数据上的损失值，用于衡量y_pred和y_true之间的差距。
4. 计算损失相对于模型参数的梯度。这一步叫作反向传播（backward pass）。
5. 将参数沿着梯度的反方向移动一小步，比如W -= learning_rate * gradient，从而使这批数据上的损失值减小一些。学习率（learning_rate）是一个调节梯度下降“速度”的标量因子。

上面介绍的方法叫作小批量随机梯度下降（mini-batch stochastic gradient descent，简称小批量SGD）。术语随机（stochastic）是指每批数据都是随机抽取的（stochastic在科学上是random的同义词）
![沿着一维损失函数曲线的随机梯度下降](image/沿着一维损失函数曲线的随机梯度下降.png)
如上图，learning_rate因子的取值很重要。如果取值太小，那么沿着曲线下降需要很多次迭代，而且可能会陷入局部极小点。如果取值过大，那么更新权重值之后可能会出现在曲线上完全随机的位置。

注意，小批量SGD算法的一个变体是每次迭代只抽取一个样本和目标，而不是抽取一批数据。这叫作真SGD（true SGD，有别于小批量SGD）。还可以走向另一个极端：每次迭代都在所有数据上运行，这叫作批量梯度下降（batch gradient descent）。这样做的话，每次更新权重都会更加准确，但计算成本也高得多。这两个极端之间有效的折中方法则是选择合理的小批量大小。

此外，SGD还有多种变体，比如带动量的SGD、Adagrad、RMSprop等。它们的不同之处在于，计算下一次权重更新时还要考虑上一次权重更新，而不是仅考虑当前的梯度值。这些变体被称为优化方法（optimization method）或优化器（optimizer）。动量的概念尤其值得关注，它被用于许多变体。动量解决了SGD的两个问题：收敛速度和局部极小值。

## 2.4.4 链式求导：反向传播算法
反向传播就是将链式法则应用于计算图，仅此而已。反向传播从最终损失值开始，自下而上反向运行，计算每个参数对损失值的贡献。这就是“反向传播”这个名称的由来：我们“反向传播”计算图中不同节点对损失值的贡献。

如今，人们利用能够自动微分的现代框架来实现神经网络，比如TensorFlow。自动微分是利用计算图来实现的。自动微分可以计算任意可微张量运算组合的梯度，只需要写出前向传播，而无须做任何额外工作。现在，多亏了现代自动微分工具，我们再也不必自己实现反向传播！